In [2]:
# Import all of the things you need to import!
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import re
from nltk.stem.porter import PorterStemmer

pd.options.display.max_columns = 30
%matplotlib inline



# Homework 14 (or so): TF-IDF text analysis and clustering

Hooray, we kind of figured out how text analysis works! Some of it is still magic, but at least the **TF** and **IDF** parts make a little sense. Kind of. Somewhat.

No, just kidding, we're *professionals* now.

## Investigating the Congressional Record

The [Congressional Record](https://en.wikipedia.org/wiki/Congressional_Record) is more or less what happened in Congress every single day. Speeches and all that. A good large source of text data, maybe?

Let's pretend it's totally secret but we just got it leaked to us in a data dump, and we need to check it out. It was leaked from [this page here](http://www.cs.cornell.edu/home/llee/data/convote.html).

In [3]:
# If you'd like to download it through the command line...
!curl -O http://www.cs.cornell.edu/home/llee/data/convote/convote_v1.1.tar.gz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 9607k  100 9607k    0     0  2273k      0  0:00:04  0:00:04 --:--:-- 2274k


In [4]:
# And then extract it through the command line...
!tar -zxf convote_v1.1.tar.gz

You can explore the files if you'd like, but we're going to get the ones from `convote_v1.1/data_stage_one/development_set/`. It's a bunch of text files.

In [5]:
# glob finds files matching a certain filename pattern
import glob

# Give me all the text files
paths = glob.glob('convote_v1.1/data_stage_one/development_set/*')
paths[:5]

['convote_v1.1/data_stage_one/development_set/052_400011_0327014_DON.txt',
 'convote_v1.1/data_stage_one/development_set/052_400011_0327025_DON.txt',
 'convote_v1.1/data_stage_one/development_set/052_400011_0327044_DON.txt',
 'convote_v1.1/data_stage_one/development_set/052_400011_0327046_DON.txt',
 'convote_v1.1/data_stage_one/development_set/052_400011_1479036_DON.txt']

In [6]:
len(paths)

702

So great, we have 702 of them. Now let's import them.

In [7]:
speeches = []
for path in paths:
    with open(path) as speech_file:
        speech = {
            'pathname': path,
            'filename': path.split('/')[-1],
            'content': speech_file.read()
        }
    speeches.append(speech)
speeches_df = pd.DataFrame(speeches)
speeches_df.head()

,content,filename,pathname
0,"mr. chairman , i thank the gentlewoman for yie...",052_400011_0327014_DON.txt,convote_v1.1/data_stage_one/development_set/05...
1,"mr. chairman , i want to thank my good friend ...",052_400011_0327025_DON.txt,convote_v1.1/data_stage_one/development_set/05...
2,"mr. chairman , i rise to make two fundamental ...",052_400011_0327044_DON.txt,convote_v1.1/data_stage_one/development_set/05...
3,"mr. chairman , reclaiming my time , let me mak...",052_400011_0327046_DON.txt,convote_v1.1/data_stage_one/development_set/05...
4,"mr. chairman , i thank my distinguished collea...",052_400011_1479036_DON.txt,convote_v1.1/data_stage_one/development_set/05...


In class we had the `texts` variable. For the homework can just do `speeches_df['content']` to get the same sort of list of stuff.

**Take a look at the contents of the first 5 speeches**

In [13]:
first_5 = speeches_df['content'].head(5)
first_5

0    mr. chairman , i thank the gentlewoman for yie...
1    mr. chairman , i want to thank my good friend ...
2    mr. chairman , i rise to make two fundamental ...
3    mr. chairman , reclaiming my time , let me mak...
4    mr. chairman , i thank my distinguished collea...
Name: content, dtype: object

# Doing our analysis

Use the `sklearn` package and a plain boring `CountVectorizer` to get a list of all of the tokens used in the speeches. If it won't list them all, that's ok! Make a dataframe with those terms as columns.

**Be sure to include English-language stopwords**

In [14]:
countvectorizer = CountVectorizer(stop_words='english')

In [16]:
tokens = countvectorizer.fit_transform(speeches_df['content'])

In [17]:
countvectorizer.get_feature_names()

['000',
 '00007',
 '018',
 '050',
 '092',
 '10',
 '100',
 '106',
 '107',
 '108',
 '108th',
 '109th',
 '10th',
 '11',
 '110',
 '114',
 '117',
 '118',
 '11th',
 '12',
 '120',
 '121',
 '122',
 '123',
 '125',
 '128',
 '12898',
 '13',
 '13279',
 '1332',
 '1335',
 '1344',
 '135',
 '138',
 '14',
 '140',
 '143',
 '144',
 '145',
 '149',
 '1498',
 '14th',
 '15',
 '150',
 '1520',
 '153',
 '155',
 '159',
 '16',
 '160',
 '162',
 '163',
 '165',
 '1671',
 '1675',
 '17',
 '170',
 '1700',
 '174',
 '178',
 '1787',
 '17th',
 '18',
 '180',
 '1800',
 '1800s',
 '181',
 '1812',
 '1855',
 '186',
 '1868',
 '18th',
 '19',
 '190',
 '1907',
 '1922',
 '1927',
 '1930',
 '1940s',
 '1950s',
 '196',
 '1960',
 '1960s',
 '1964',
 '1965',
 '1967',
 '1970s',
 '1971',
 '1972',
 '1973',
 '1974',
 '1976',
 '1979',
 '198',
 '1980s',
 '1981',
 '1982',
 '1983',
 '1984',
 '1985',
 '1986',
 '1987',
 '1988',
 '1989',
 '1990',
 '1990s',
 '1991',
 '1992',
 '1993',
 '1994',
 '1995',
 '1996',
 '1997',
 '1998',
 '1999',
 '19th',
 '1st'

Okay, it's **far** too big to even look at. Let's try to get a list of features from a new `CountVectorizer` that only takes the top 100 words.

In [19]:
countvectorizer_new = CountVectorizer(max_features=100, stop_words='english')

In [20]:
tokens_new = countvectorizer_new.fit_transform(speeches_df['content'])


In [57]:
tokens_complete = pd.DataFrame(tokens_new.toarray(), columns=countvectorizer_new.get_feature_names())

Now let's push all of that into a dataframe with nicely named columns.

In [23]:
only_100_tokens = pd.DataFrame(tokens_new.toarray(), columns=countvectorizer_new.get_feature_names())
only_100_tokens

,000,11,act,allow,amendment,america,american,amp,association,balance,based,believe,bipartisan,chairman,children,...,teachers,thank,think,time,today,trade,united,urge,vote,want,way,work,year,years,yield
0,0,1,3,0,0,0,3,0,0,0,0,1,0,3,0,...,0,1,3,3,2,0,1,0,0,1,1,0,0,0,1
1,0,0,1,1,1,0,0,0,0,1,0,0,0,2,0,...,0,1,0,2,2,0,0,0,1,1,3,0,1,0,0
2,0,0,0,0,0,0,1,0,0,0,0,0,0,2,0,...,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1
3,0,0,0,0,0,1,0,0,0,1,0,0,0,2,0,...,0,0,0,2,0,0,1,0,1,1,1,0,0,0,0
4,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,...,0,1,0,1,0,0,0,0,2,0,0,0,0,0,2
5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
6,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,...,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
8,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,...,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0
9,0,0,0,1,0,0,1,0,0,0,0,0,0,2,0,...,0,0,2,0,0,0,1,0,0,0,0,0,0,0,0


Everyone seems to start their speeches with "mr chairman" - how many speeches are there total, and many don't mention "chairman" and how many mention neither "mr" nor "chairman"?

In [25]:
only_100_tokens.describe()
#There are 702 speeches in this Dataframe

,000,11,act,allow,amendment,america,american,amp,association,balance,based,believe,bipartisan,chairman,children,...,teachers,thank,think,time,today,trade,united,urge,vote,want,way,work,year,years,yield
count,702.000000,702.000000,702.000000,702.000000,702.000000,702.000000,702.000000,702.000000,702.000000,702.000000,702.000000,702.000000,702.000000,702.000000,702.000000,...,702.000000,702.000000,702.000000,702.000000,702.000000,702.000000,702.000000,702.000000,702.000000,702.000000,702.000000,702.000000,702.000000,702.000000,702.000000
mean,0.205128,0.233618,0.441595,0.219373,1.190883,0.222222,0.407407,0.465812,0.407407,0.267806,0.309117,0.220798,0.186610,1.247863,0.666667,...,0.212251,0.280627,0.313390,1.116809,0.360399,0.531339,0.294872,0.254986,0.418803,0.336182,0.233618,0.260684,0.321937,0.273504,0.574074
std,1.210075,1.841697,1.934411,0.897148,3.827676,1.026978,1.759215,10.931813,8.509915,0.520140,2.302365,0.868514,1.417515,1.558590,3.436400,...,2.541348,0.706316,0.899401,1.779749,0.882400,3.117483,1.529529,1.548713,1.291292,0.876595,0.687868,0.899592,0.875439,0.956610,0.679481
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,...,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000,0.000000,...,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
max,25.000000,44.000000,35.000000,18.000000,87.000000,17.000000,26.000000,288.000000,224.000000,4.000000,56.000000,14.000000,35.000000,18.000000,74.000000,...,65.000000,9.000000,9.000000,19.000000,9.000000,63.000000,31.000000,39.000000,23.000000,9.000000,9.000000,12.000000,7.000000,16.000000,4.000000


In [38]:
only_100_tokens['chairman_less'] = only_100_tokens['chairman'] == 0
only_100_tokens['chairman_less'].describe()

count       702
unique        2
top       False
freq        452
Name: chairman_less, dtype: object

In [39]:
print('There are',702-452,'Speeches withou the word chairman in it')

There are 250 Speeches withou the word chairman in it


In [40]:
only_100_tokens['mr_less'] = only_100_tokens['mr'] == 0
only_100_tokens['mr_less'].describe()

count       702
unique        2
top       False
freq        623
Name: mr_less, dtype: object

In [41]:
print('There are',702-623,'Speeches withou the word chairman in it')

There are 79 Speeches withou the word chairman in it


What is the index of the speech thank is the most thankful, a.k.a. includes the word 'thank' the most times?

In [47]:
only_100_tokens['thank'].sort_values(ascending=False).head(1)

577    9
Name: thank, dtype: int64

If I'm searching for `China` and `trade`, what are the top 3 speeches to read according to the `CountVectoriser`?

In [46]:
only_100_tokens['china trade'] = only_100_tokens['china'] + only_100_tokens['trade']
only_100_tokens['china trade'].sort_values(ascending=False).head(3)

379    92
399    36
345    27
Name: china trade, dtype: int64

Now what if I'm using a `TfidfVectorizer`?

**What's the content of the speeches?** Here's a way to get them:

In [48]:
# index 0 is the first speech, which was the first one imported.
paths[0]

'convote_v1.1/data_stage_one/development_set/052_400011_0327014_DON.txt'

In [49]:
# Pass that into 'cat' using { } which lets you put variables in shell commands
# that way you can pass the path to cat
!cat {paths[0]}

mr. chairman , i thank the gentlewoman for yielding me this time . 
my good colleague from california raised the exact and critical point . 
the question is , what happens during those 45 days ? 
we will need to support elections . 
there is not a single member of this house who has not supported some form of general election , a special election , to replace the members at some point . 
but during that 45 days , what happens ? 
the chair of the constitution subcommittee says this is what happens : martial law . 
we do not know who would fill the vacancy of the presidency , but we do know that the succession act most likely suggests it would be an unelected person . 
the sponsors of the bill before us today insist , and i think rightfully so , on the importance of elections . 
but to then say that during a 45-day period we would have none of the checks and balances so fundamental to our constitution , none of the separation of powers , and that the presidency would be filled by an unel

**Now search for something else!** Another two terms that might show up. `elections` and `chaos`? Whatever you thnik might be interesting.

In [67]:
tokens_complete['america'] = tokens_complete['america'].sort_values(ascending=False) >= 1
tokens_complete['america'].describe()

count       702
unique        2
top       False
freq        627
Name: america, dtype: object

In [70]:
tokens_complete[tokens_complete['america'] == True].count().head(1)
#wow! Someone said america 75 times in an speech!

000    75
dtype: int64

In [71]:
tokens_complete['elections'] = tokens_complete['elections'].sort_values(ascending=False) >= 1
tokens_complete['elections'].describe()
# the word elections has been present in nearly all speeches! 

count       702
unique        2
top       False
freq        664
Name: elections, dtype: object

# Enough of this garbage, let's cluster

Using a **simple counting vectorizer**, cluster the documents into **eight categories**, telling me what the top terms are per category.

Using a **term frequency vectorizer**, cluster the documents into **eight categories**, telling me what the top terms are per category.

Using a **term frequency inverse document frequency vectorizer**, cluster the documents into **eight categories**, telling me what the top terms are per category.

In [ ]:
## Im still working on this one!

**Which one do you think works the best?**

# Harry Potter time

I have a scraped collection of Harry Potter fanfiction at https://github.com/ledeprogram/courses/raw/master/algorithms/data/hp.zip.

I want you to read them in, vectorize them and cluster them. Use this process to find out **the two types of Harry Potter fanfiction**. What is your hypothesis?